In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy as signal
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
data = pd.read_csv('emotions.csv')
data.head()



,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [4]:
#convert labels to numerical values
label_mapping = {'NEGATIVE' : 0, 'NEUTRAL' : 1, 'POSITIVE': 2}
data['label'] = data['label'].map(label_mapping)

In [5]:
#normalization
scaler = StandardScaler()
data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])

In [6]:
#splitting in test train set
X = data.drop('label', axis = 1)
Y = data['label']
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 123)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [8]:
#compiling
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(x_train, y_train, validation_split=0.2, epochs=70, batch_size=32, verbose=2)

Epoch 1/70


38/38 - 2s - loss: 1.0727 - accuracy: 0.7695 - val_loss: 0.5297 - val_accuracy: 0.9030 - 2s/epoch - 52ms/step
Epoch 2/70
38/38 - 0s - loss: 0.6976 - accuracy: 0.8650 - val_loss: 0.2504 - val_accuracy: 0.9398 - 330ms/epoch - 9ms/step
Epoch 3/70
38/38 - 0s - loss: 0.5958 - accuracy: 0.8843 - val_loss: 0.4668 - val_accuracy: 0.9197 - 348ms/epoch - 9ms/step
Epoch 4/70
38/38 - 0s - loss: 0.5335 - accuracy: 0.8944 - val_loss: 0.2395 - val_accuracy: 0.9431 - 304ms/epoch - 8ms/step
Epoch 5/70
38/38 - 0s - loss: 0.2909 - accuracy: 0.9061 - val_loss: 0.3027 - val_accuracy: 0.9231 - 327ms/epoch - 9ms/step
Epoch 6/70
38/38 - 0s - loss: 0.3281 - accuracy: 0.9028 - val_loss: 0.2397 - val_accuracy: 0.9298 - 319ms/epoch - 8ms/step
Epoch 7/70
38/38 - 0s - loss: 0.2314 - accuracy: 0.9329 - val_loss: 0.2673 - val_accuracy: 0.9197 - 318ms/epoch - 8ms/step
Epoch 8/70
38/38 - 0s - loss: 0.2309 - accuracy: 0.9296 - val_loss: 0.2444 - val_accuracy: 0.9298 - 320ms/epoch - 8ms/step
Epoch 9/70
38/38

In [10]:
#evaluation
model_acc = model.evaluate(x_test, y_test, verbose = 0)[1]
print(f'{model_acc*100} %')

96.5624988079071 %


In [13]:
#make pred
y_pred = np.argmax(model.predict(x_test), axis = 1)


20/20 [==============================] - 0s 3ms/step
[0 0 2 1 0 2 0 1 1 2 1 1 0 2 2 2 1 1 0 1 1 2 0 1 0 2 0 2 1 2 1 1 2 1 1 1 1
 1 2 0 1 1 2 1 0 1 0 1 2 1 2 2 1 1 0 1 2 0 2 2 2 1 2 1 0 2 1 2 1 2 1 0 1 1
 1 2 0 2 1 1 2 0 0 2 2 0 2 1 2 0 2 1 2 1 1 0 1 1 1 2 0 2 0 0 2 1 1 2 0 1 2
 1 2 2 2 2 2 1 0 0 2 0 2 0 1 1 1 0 2 0 2 0 2 1 0 2 0 0 0 0 0 2 2 0 0 0 1 0
 2 1 0 2 2 1 0 1 2 2 1 2 0 2 2 2 0 0 1 0 0 2 1 0 0 1 0 1 1 2 1 1 2 1 1 1 0
 2 1 2 2 2 1 0 1 0 0 2 2 0 2 2 2 1 1 1 0 0 0 0 2 2 1 2 0 1 2 0 0 0 1 0 1 2
 1 2 0 1 0 1 0 0 1 0 1 0 0 1 2 2 2 0 2 2 2 0 2 2 0 1 2 0 1 2 1 0 1 0 1 1 0
 1 2 1 1 0 1 1 1 1 0 1 2 0 2 2 0 1 0 1 2 2 1 1 1 0 1 1 2 0 2 2 2 0 0 2 2 0
 0 2 2 1 1 2 0 2 1 2 2 1 1 0 0 2 0 0 0 2 1 1 1 1 1 1 2 0 1 0 1 1 0 1 0 0 0
 0 2 1 1 1 1 0 0 2 2 2 2 2 1 1 1 0 2 2 1 2 1 0 0 0 0 0 0 1 2 2 2 1 1 1 1 1
 2 2 0 1 0 1 2 1 1 2 1 1 1 0 0 1 1 2 0 1 2 1 0 2 0 1 2 2 0 1 2 0 1 1 1 1 1
 1 0 2 1 2 2 1 1 2 1 1 0 1 2 2 0 1 0 0 1 0 2 0 0 0 1 1 2 0 0 2 0 1 0 0 1 2
 0 1 0 2 1 1 0 1 1 1 0 1 2 2 0 2 2 0 1 2 0 0 1 

In [12]:
joblib.dump(model, 'my_model.pkl')

['my_model.pkl']